In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import pandas as pd
import seaborn as sn
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix, accuracy_score, f1_score
cudnn.benchmark = True
plt.ion()   # interactive mode

In [2]:
# use GPU when available
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
# Print GPU name
print('GPU: ', torch.cuda.get_device_name(device=device))
#number of usable CPU cores
core_count = len(os.sched_getaffinity(0)) #os.cpu_count() for local windows machine
print('CPU cores: ', core_count)
# current working directory
wd = os.getcwd()
print('Working directory: ', wd)

# start time
t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)
print(current_time)

GPU:  NVIDIA A100-SXM4-80GB
CPU cores:  2
Working directory:  /blue/eee4773/gmarais/Transfer_learning/param_optim
10:37:38


In [3]:
def train_model(model, criterion, optimizer, scheduler, dataloaders, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    train_dict = {'train':[[],[]], 'val':[[],[]]}

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                labels = labels.type(torch.LongTensor) 
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            train_dict[phase][0].append(epoch_loss)
            train_dict[phase][1].append(epoch_acc.cpu().item())

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, train_dict

In [4]:
def train(dataloaders, epochs, gamma, step_size, learning_rate, class_num=10):
    # load model
    model_conv = torchvision.models.efficientnet_v2_l(weights='EfficientNet_V2_L_Weights.IMAGENET1K_V1')
    for param in model_conv.parameters():
        param.requires_grad = False
    # Parameters of newly constructed modules have requires_grad=True by default
    num_ftrs = model_conv.classifier[1].in_features
    model_conv.classifier = nn.Linear(num_ftrs, class_num)
    model_conv = model_conv.to(device)
    criterion = nn.CrossEntropyLoss()
    # Observe that only parameters of final layer are being optimized
    optimizer_conv = optim.Adam(model_conv.classifier.parameters(), lr=learning_rate)
    # Decay LR by a factor of 0.1 every 7 epochs
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=step_size, gamma=gamma)
    # train model
    best_model, train_dict_fixed = train_model(model_conv, 
                                               criterion, 
                                               optimizer_conv, 
                                               exp_lr_scheduler, 
                                               num_epochs=epochs,
                                               dataloaders=dataloaders)
    return best_model, train_dict_fixed

In [5]:
def test(best_model, dataloader, class_num=10):
    model = torchvision.models.efficientnet_v2_l(weights='EfficientNet_V2_L_Weights.IMAGENET1K_V1')
    num_ftrs = model.classifier[1].in_features
    model.classifier = torch.nn.Linear(num_ftrs, class_num)
    model.load_state_dict(best_model.state_dict())
    model.to(device)
    model.eval()
    model.train(mode=False)
    
    cm=0
    cm_multi=0
    all_labels_lst = []
    all_preds_lst = []
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)
            labels = labels.to(int)
            multi_labels_lst = torch.nn.functional.one_hot(labels, num_classes=class_num).cpu().tolist()
            labels_lst = labels.cpu().tolist()
            all_labels_lst.append(labels_lst)
            

            outputs = model(inputs)
            probabilities = torch.nn.functional.softmax(outputs, dim=1)
            multi_preds_lst = (probabilities >= 0.9).type(torch.uint8).cpu().tolist()
            preds_lst = torch.argmax(probabilities, axis=1).cpu().tolist()
            all_preds_lst.append(preds_lst)

            cm += confusion_matrix(y_true=labels_lst, y_pred=preds_lst, labels=[0,1,2,3,4,5,6,7,8,9])
            cm_multi += multilabel_confusion_matrix(y_true=multi_labels_lst, y_pred=multi_preds_lst)
            
    all_labels_lst = sum(all_labels_lst, [])
    all_preds_lst = sum(all_preds_lst, [])
    print('Acc: ', accuracy_score(all_labels_lst, all_preds_lst))
    print('Micro F1: ', f1_score(all_labels_lst, all_preds_lst, average='micro'))
    
    return cm, cm_multi

In [ ]:
# Loading Data
data = np.load(wd+'/nodup_data.npy')
labels = np.load(wd+'/nodup_labels.npy')

# reshape data
reshaped_data = data.reshape((300,300,3,data.shape[1]))
reshaped_data = np.moveaxis(reshaped_data, source=[0, 1, 2, 3], destination=[2, 3, 1, 0])
labels = np.array(labels,dtype=int)

# label names
class_names = ['Stop','Yield','Red Light','Green Light','Roundabout','Right Turn Only',
                'Do Not Enter','Crosswalk','Handicap Parking','No Parking']

#######################################################################################
# # parameter ranges
# kfolds = 5
# epoch_count = 15
# batch_size_lst = [1, 32, 64]
# gamma_lst = [0.1, 0.2, 0.3]
# step_size_lst = [1, 9, 15]
# lr_lst = [0.001,0.01, 0.1]

# parameter ranges
kfolds = 5
epoch_count = 15
batch_size_lst = [64, 128, 256, 512]
gamma_lst = [0.001, 0.01, 0.1, 0.2, 0.3]
step_size_lst = [1, 3, 5, 7]
lr_lst = [0.001, 0.005,0.01, 0.1]
#######################################################################################

# split data into folds
kf = KFold(n_splits=kfolds, random_state=42, shuffle=True)
fold = 0
for train_index, test_index in kf.split(reshaped_data):
    x_train, x_test = reshaped_data[train_index], reshaped_data[test_index]
    t_train, t_test = labels[train_index], labels[test_index]
    x_train_te = torch.Tensor(x_train)
    x_val_te = torch.Tensor(x_test)
    t_train_te = torch.Tensor(t_train)
    t_val_te = torch.Tensor(t_test)
    fold += 1
    
    for batch_size in batch_size_lst:
        # turn tensors into data loaders
        image_datasets = {'train': TensorDataset(x_train_te,t_train_te), 'val': TensorDataset(x_val_te,t_val_te)}
        dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], 
                                                      batch_size=batch_size, 
                                                      shuffle=True, 
                                                      num_workers=core_count) for x in ['train', 'val']}
        dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
        print('Data shapes',x_train_te.shape, x_val_te.shape, t_train_te.shape, t_val_te.shape)
        
        for gamma in gamma_lst:
            for step_size in step_size_lst:
                for learning_rate in lr_lst:
                    file_name = 'multi_'+str(fold)+'_'+str(batch_size)+'_'+str(gamma)+'_'+str(step_size)+'_'+str(learning_rate)+'_cm_train.npy'
                    if file_name in os.listdir():
                        print('Already Exists')
                    else:
                        print('fold-batch_size-gamma-step_size-learning_rate', fold, batch_size, gamma, step_size, learning_rate)
                        best_model, train_dict_fixed = train(dataloaders=dataloaders, 
                                                             epochs=epoch_count, 
                                                             gamma=gamma, 
                                                             step_size=step_size, 
                                                             learning_rate=learning_rate, 
                                                             class_num=10)

                        # save training epoch data to disk
                        df = pd.DataFrame()
                        df['train_loss'] = train_dict_fixed['train'][0]
                        df['val_loss'] = train_dict_fixed['val'][0]
                        df['train_acc'] = train_dict_fixed['train'][1]
                        df['val_acc'] = train_dict_fixed['val'][1]
                        df.to_csv(wd+'/train_'+str(fold)+'_'+str(batch_size)+'_'+str(gamma)+'_'+str(step_size)+'_'+str(learning_rate)+'.csv')

                        print('Val')
                        cm, cm_multi = test(best_model=best_model, dataloader=dataloaders['val'])
                        cm_df = pd.DataFrame(cm)
                        cm_df.to_csv(wd+'/'+str(fold)+'_'+str(batch_size)+'_'+str(gamma)+'_'+str(step_size)+'_'+str(learning_rate)+'_cm_val.csv')
                        np.save(wd+'/multi_'+str(fold)+'_'+str(batch_size)+'_'+str(gamma)+'_'+str(step_size)+'_'+str(learning_rate)+'_cm_val', cm_multi)

                        print('Train')
                        cm, cm_multi = test(best_model=best_model, dataloader=dataloaders['train'])
                        cm_df = pd.DataFrame(cm)
                        cm_df.to_csv(wd+'/'+str(fold)+'_'+str(batch_size)+'_'+str(gamma)+'_'+str(step_size)+'_'+str(learning_rate)+'_cm_train.csv')
                        np.save(wd+'/multi_'+str(fold)+'_'+str(batch_size)+'_'+str(gamma)+'_'+str(step_size)+'_'+str(learning_rate)+'_cm_train', cm_multi)

                        # end time 
                        t = time.localtime()
                        current_time = time.strftime("%H:%M:%S", t)
                        print('fold-batch_size-gamma-step_size-learning_rate', fold, batch_size, gamma, step_size, learning_rate)
                        print('Done at: ', current_time)
                        print('_____________________________________________\n\n')
print('FINAL FINISHED')
print('_____________________________________________\n\n')

Data shapes torch.Size([4904, 3, 300, 300]) torch.Size([1226, 3, 300, 300]) torch.Size([4904]) torch.Size([1226])
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
Already Exists
A